# <center><u> Final LSTM/GRU Layout - Adri and Elian

In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 825.2 kB/s eta 0:00:00


In [2]:
!pip install python-dotenv

In [3]:
!pip install psycopg2

In [5]:
%cd '/content/drive/MyDrive/AML GRU Second 15 Houses'

/content/drive/MyDrive/AML GRU Second 15 Houses


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Define Inputs

In [6]:
n_input = 144
n_features = 5
n_output = 72
n_split = 2 * 24 * 324

# Get Household Data

In [7]:
#Import Libraries and Packages
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2
from dotenv import load_dotenv
load_dotenv()

def get_household_data(household):
    #Access Credentials
    # db_host = os.getenv("DB_HOST")
    # db_name = os.getenv("DB_NAME")
    # db_user = os.getenv("DB_USER")
    # db_password = os.getenv("DB_PASSWORD")
    # db_port = os.getenv("DB_PORT")

    db_host="aml-esda-db.postgres.database.azure.com"
    db_name="postgres"
    db_user="elian"
    db_password="2xKuZG~oP2v9"
    db_port="5432"

    conn = psycopg2.connect(
    host=db_host,
    dbname=db_name,
    user=db_user,
    password=db_password,
    port=db_port
    )

    query=f"""
    select *
    from agg.tidy_data_final
    where site = {household}
    """

    df = pd.read_sql_query(query, conn)
    df = df.set_index('timestamp', drop=False)
    df = df.sort_index()

    return df


# Prepare Data Set for LSTM/GRU Models

In [8]:
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()

def data_prep(df):
    global n_input
    global n_features
    global n_output
    global n_split

    # Select features of use for later on
    df_input = df[['net_load', 'precipitation_probability',
                'solar_radiation','sunshine_duration','weekend_or_bank_holiday',
                    'month', 'day', 'hour', 'day_of_week', 'season',
                    'avg_net_load',]]


    variables = ['net_load','solar_radiation','sunshine_duration', 'precipitation_probability', 'avg_net_load' ]


    for var in variables:
        new_var_name = var + '_norm'
        df_input[new_var_name] = scaler.fit_transform(df_input[[var]])

    df_input['sunshine_duration_norm(t+48)'] = df_input['sunshine_duration_norm'].shift(-48) #Add prediction for time the day after
    df_input['solar_radiation_norm(t+48)'] = df_input['solar_radiation_norm'].shift(-48) #Add prediction for time the day after
    df_input['precipitation_probability_norm(t+48)'] = df_input['precipitation_probability_norm'].shift(-48) #Add prediction for time the day after

    df_input.dropna(inplace=True)

    # df_input = df_input[['net_load_norm', 'avg_net_load_norm', 'weekend_or_bank_holiday', 'sunshine_duration_norm(t+48)',
    #            'solar_radiation_norm(t+48)','precipitation_probability_norm(t+48)',
    #            'month', 'day', 'hour', 'day_of_week', 'season',
    #                 ]]

    df_input = df_input[['net_load_norm', 'weekend_or_bank_holiday', 'sunshine_duration_norm(t+48)',
            'solar_radiation_norm(t+48)','precipitation_probability_norm(t+48)'
                ]]

    X, y = [], []
    for i in range(len(df_input) - n_input - n_output + 1):
        # Select all columns for the input sequence
        X.append(df_input.iloc[i:(i + n_input)].values)

        y.append(df_input['net_load_norm'].iloc[(i + n_input):(i + n_input + n_output)].values)

    X = np.array(X).reshape((len(X), n_input, -1))
    y = np.array(y)

    X_train = X[:n_split]
    y_train = y[:n_split]

    X_test = X[n_split:]
    y_test = y[n_split:]

    return X_train, y_train, X_test, y_test, df_input

# Build and Fit LSTM/ GRU Models

In [9]:
# Import all Necessary Keras Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

def build_model(X_train, y_train, model_name):
    model = Sequential()

    model.add(GRU(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True ))
    # model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))

    model.add(GRU(32))
    # model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.01))

    model.add(Dropout(0.2))
    model.add(Dense(y_train.shape[1]))

    optimizer = Adam(learning_rate=0.001)

    #Compile Model
    cp1 = ModelCheckpoint(f'{model_name}.keras', save_best_only=True)
    model.compile(loss='mse', optimizer=optimizer,  metrics=[MeanSquaredError(), MeanAbsoluteError()])

    #Fit Model to Training Data with 10% Validation Split
    history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_split=0.1, callbacks=[cp1])

    return model

# Get Training Outputs

### Evaluation Metrics

In [10]:
def get_train_metrics(model, X_train, y_train):
    _, train_mse, train_mae = model.evaluate(X_train, y_train, verbose=0)

    return train_mse, train_mae

### Visualisation

In [11]:
def get_training_viz(start_section, end_section, df_input, X_train, y_train, model):
    global n_input
    global n_features
    global n_output
    global n_split



    df_ts_train = df_input[n_input:n_split]

    X_train_shift = X_train[24:]
    y_train_shift = y_train[24:]

    X_train_input = X_train_shift[::48]
    y_train_input = y_train_shift[::48]


    predictions = []

    for i in X_train_input[start_section:end_section]:
        i_reshaped = i.reshape(1, n_input, n_features)

        prediction = model.predict(i_reshaped)

        # We want to omit the first 12 hours the predictions i.e., the non-shaded squares in the figure above.
        prediction_list = list(prediction[0][24:])
        predictions.append(prediction_list)


    y_train_input_24h = []

    for i in y_train_input[start_section:end_section]:
        # i_reshaped = i.reshape(1, n_input, n_features)
        y_train_input_24h.append(i[24:])


    plt.figure(figsize=(12,4))
    plt.title(f"Date Range: {df_ts_train.index[start_section*48+48].strftime('%d/%m/%Y')} - {df_ts_train.index[end_section*48+48].strftime('%d/%m/%Y')}")

    # We want to plot the predictions from 00:00 to 00:00 the next day.
    plt.plot(df_ts_train.index[start_section*48+48: end_section*48+48], np.array(predictions).flatten(), label='Prediction')
    plt.plot(df_ts_train.index[start_section*48+48: end_section*48+48], np.array(y_train_input_24h).flatten(), label = 'Actual')
    plt.xlabel('Datetime')
    plt.ylabel('Normalised Net Load (-)')


    # Generate vertical lines for when we predict
    xcoords_DAM = df_ts_train.index[start_section*48+72: (end_section)*48+72][::48]

    for i, xc in enumerate(xcoords_DAM):
        if i == 0:  # First item gets the label
            plt.axvline(x=xc, color='red', linestyle='--', alpha=0.5, label='DAM Gate Closure')
        else:  # Subsequent items do not
            plt.axvline(x=xc, color='red', linestyle='--', alpha=0.5)



    # Generate vertical lines for DAM closure
    xcoords_pred = df_ts_train.index[start_section*48+48: (end_section+1)*48+48][::48]

    for i, xc in enumerate(xcoords_pred):
        if i == 0:  # First item gets the label
            plt.axvline(x=xc, color='purple', linestyle='--', alpha=0.3, label='Energy Delivery Prediction')
        else:  # Subsequent items do not
            plt.axvline(x=xc, color='purple', linestyle='--', alpha=0.3)


    plt.legend(bbox_to_anchor=(0.5, 1.15), loc='center', borderaxespad=0., ncol=4)

# Get Test Outputs

### Evaluation Metrics

In [12]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

def get_test_metrics(model, X_test, y_test):
    global n_input
    global n_features
    global n_output
    global n_split

    X_test_shift = X_test[24:]
    y_test_shift = y_test[24:]

    X_test_input = X_test_shift[::48]
    y_test_input = y_test_shift[::48]

    predictions = []

    for i in X_test_input:
        i_reshaped = i.reshape(1, n_input, n_features)

        prediction = model.predict(i_reshaped, verbose=0)

        # We want to omit the first 12 hours the predictions i.e., the non-shaded squares in the figure above.
        prediction_list = list(prediction[0][24:])
        predictions.append(prediction_list)


    y_test_input_24h = []

    for i in y_test_input:
        # i_reshaped = i.reshape(1, n_input, n_features)
        y_test_input_24h.append(i[24:])

    # Flatten the lists of lists into single lists
    predictions_flat = [item for sublist in predictions for item in sublist]
    y_test_flat = [item for sublist in y_test_input_24h for item in sublist]

    # Calculate MSE
    test_mse = mean_squared_error(y_test_flat, predictions_flat)

    # Calculate MAE
    test_mae = mean_absolute_error(y_test_flat, predictions_flat)

    return test_mse, test_mae

### Visualisation

In [13]:
def get_testing_viz(start_section, end_section, df_input, X_test, y_test, model):
    global n_input
    global n_features
    global n_output
    global n_split

    df_ts_test = df_input[n_input+n_split:]

    X_test_shift = X_test[24:]
    y_test_shift = y_test[24:]

    X_test_input = X_test_shift[::48]
    y_test_input = y_test_shift[::48]


    predictions = []

    for i in X_test_input[start_section:end_section]:
        i_reshaped = i.reshape(1, n_input, n_features)

        prediction = model.predict(i_reshaped)

        # We want to omit the first 12 hours the predictions i.e., the non-shaded squares in the figure above.
        prediction_list = list(prediction[0][24:])
        predictions.append(prediction_list)


    y_test_input_24h = []

    for i in y_test_input[start_section:end_section]:
        # i_reshaped = i.reshape(1, n_input, n_features)
        y_test_input_24h.append(i[24:])


    plt.figure(figsize=(12,4))
    plt.title(f"Date Range: {df_ts_test.index[start_section*48+48].strftime('%d/%m/%Y')} - {df_ts_test.index[end_section*48+48].strftime('%d/%m/%Y')}")

    # We want to plot the predictions from 00:00 to 00:00 the next day.
    plt.plot(df_ts_test.index[start_section*48+48: end_section*48+48], np.array(predictions).flatten(), label='Prediction')
    plt.plot(df_ts_test.index[start_section*48+48: end_section*48+48], np.array(y_test_input_24h).flatten(), label = 'Actual')
    plt.xlabel('Datetime')
    plt.ylabel('Normalised Net Load (-)')


    # Generate vertical lines for when we predict
    xcoords_DAM = df_ts_test.index[start_section*48+72: (end_section)*48+72][::48]

    for i, xc in enumerate(xcoords_DAM):
        if i == 0:  # First item gets the label
            plt.axvline(x=xc, color='red', linestyle='--', alpha=0.5, label='DAM Gate Closure')
        else:  # Subsequent items do not
            plt.axvline(x=xc, color='red', linestyle='--', alpha=0.5)



    # Generate vertical lines for DAM closure
    xcoords_pred = df_ts_test.index[start_section*48+48: (end_section+1)*48+48][::48]

    for i, xc in enumerate(xcoords_pred):
        if i == 0:  # First item gets the label
            plt.axvline(x=xc, color='purple', linestyle='--', alpha=0.3, label='Energy Delivery Prediction')
        else:  # Subsequent items do not
            plt.axvline(x=xc, color='purple', linestyle='--', alpha=0.3)


    plt.legend(bbox_to_anchor=(0.5, 1.15), loc='center', borderaxespad=0., ncol=4)

# Hyperparameter tuning

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, LeakyReLU, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import MeanSquaredError, MeanAbsoluteError
from tensorflow.keras.callbacks import ModelCheckpoint
from kerastuner.tuners import RandomSearch

def build_model_hyperparm(hp):
    model = Sequential()

    model.add(GRU(hp.Int('units_1', min_value=32, max_value=256, step=32),
                   input_shape=(X_train.shape[1], X_train.shape[2]),
                   return_sequences=hp.Choice('return_sequences', [True, False])))
    model.add(LeakyReLU(alpha=0.01))

    if hp.Choice('return_sequences', [True, False]):
        model.add(GRU(hp.Int('units_2', min_value=32, max_value=256, step=32)))
        model.add(LeakyReLU(alpha=0.01))

    model.add(Dropout(hp.Float('dropout', min_value=0.1, max_value=0.3, step=0.1)))
    model.add(Dense(y_train.shape[1]))

    model.compile(loss='mse',
                  optimizer=Adam(learning_rate=0.001),
                  metrics=[MeanSquaredError(), MeanAbsoluteError()])

    return model

<ipython-input-14-0c699db5c18e>:6: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


# For Loop

In [15]:
import time
import warnings

warnings.filterwarnings('ignore')

households = [
    2, 3, 6, 9, 11, 12, 16, 17, 20, 21, 22, 23, 25, 28, 29, 30, 31, 33, 34, 36, 39,
    41, 42, 46, 49, 50, 53, 57, 61, 62, 64, 73, 77, 81, 84, 85, 90, 91, 94, 98, 100
]

list_train_MSE = []
list_train_MAE = []
list_test_MSE = []
list_test_MAE = []
list_first_layer = []
list_second_layer = []
list_num_layers = []
list_dropoute_rate = []
list_time_to_run = []

for household in households[15:30]:
    start_time = time.time()  # Record the start time of the loop iteration

    df = get_household_data(household)
    X_train, y_train, X_test, y_test, df_input = data_prep(df)

    # Define model checkpoint
    cp1 = ModelCheckpoint(f'house{household}.keras', save_best_only=True)

    # Initialise Random Search
    tuner = RandomSearch(
        build_model_hyperparm,
        objective='val_loss',
        max_trials=5,
        executions_per_trial=1,
        directory='model_tuning_v2',
        project_name=f'house{household}'
    )

    tuner.search(X_train, y_train, epochs=5, batch_size=16, validation_split=0.1, callbacks=[cp1], verbose=1)

    # Get the best hyperparameters
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

    print(f"""
    Optimal Hyperparameters
        1st Layer Units: {best_hps.get('units_1')}
        2nd Layer Units: {best_hps.get('units_2')}
        No. of Layers: {1 if best_hps.get('return_sequences') == 0 else 2 }
        Dropout Rate: {best_hps.get('dropout') }
    """)

    # Get the best model
    best_model = tuner.get_best_models(num_models=1)[0]
    train_mse, train_mae = get_train_metrics(best_model, X_train, y_train)
    test_mse, test_mae = get_test_metrics(best_model, X_test, y_test)


    list_train_MSE.append(train_mse)
    list_train_MAE.append(train_mae)
    list_test_MSE.append(test_mse)
    list_test_MAE.append(test_mae)
    list_first_layer.append(best_hps.get('units_1'))
    list_second_layer.append(best_hps.get('units_2'))
    list_num_layers.append(1 if best_hps.get('return_sequences') == 0 else 2 )
    list_dropoute_rate.append(best_hps.get('dropout'))

    end_time = time.time()  # Record the end time of the loop iteration
    duration = end_time - start_time  # Calculate the duration

    list_time_to_run.append(duration)

    print(f"=========================HOUSE{household}=========================")
    print(f"Train MSE: {train_mse}")
    print(f"Train MAE: {train_mae}")
    print(f"Test MSE: {test_mse}")
    print(f"Test MAE: {test_mae}")
    print(f"First Layer: {best_hps.get('units_1')}")
    print(f"Second Layer: {best_hps.get('units_2')}")
    print(f"Number of Layers: {1 if best_hps.get('return_sequences') == 0 else 2 }")
    print(f"Dropout Rate: {best_hps.get('dropout')}")
    print(f"Time Taken: {duration}")
    print(f"=========================HOUSE{household}=========================")
    print("")

data = {
    'Household': households[15:30],
    'Train MSE': list_train_MSE,
    'Train MAE': list_train_MAE,
    'Test MSE': list_test_MSE,
    'Test MAE': list_test_MAE,
    'First Layer': list_first_layer,
    'Second Layer': list_second_layer,
    'Num Layers': list_num_layers,
    'Dropout Rate': list_dropoute_rate,
    'Time to Run': list_time_to_run
}


df = pd.DataFrame(data)

df.to_csv('GRU_results_second_15.csv', index=False)

Trial 5 Complete [00h 01m 05s]
val_loss: 0.011646231636404991

Best val_loss So Far: 0.011646231636404991
Total elapsed time: 00h 05m 34s

    Optimal Hyperparameters
        1st Layer Units: 64
        2nd Layer Units: 128
        No. of Layers: 2
        Dropout Rate: 0.1
    
=========================HOUSE62=========================
Train MSE: 0.008943152613937855
Train MAE: 0.06005615368485451
Test MSE: 0.015453363945637967
Test MAE: 0.08870341249527006
First Layer: 64
Second Layer: 128
Number of Layers: 2
Dropout Rate: 0.1
Time Taken: 348.3961389064789
=========================HOUSE62=========================

